<a href="https://colab.research.google.com/github/sameekshya1999/Sleep-Stage-Classification-Using-Deep-Learning-CNN-vs.-EEGNet-Attention-/blob/main/MC%26Ensemble_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, constraints
from tensorflow.keras.utils import Sequence
import mne
import urllib.request
import os
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import gc

# Note: Removed tensorflow-probability dependency as Variational Inference is excluded
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

warnings.filterwarnings("ignore", category=DeprecationWarning)
mne.set_log_level('ERROR')

NUM_SUBJECTS = 20
NUM_NIGHTS = 2
BASE_URL = "https://physionet.org/files/sleep-edfx/1.0.0/"
TARGET_CHANNELS = ['EEG Fpz-Cz', 'EEG Pz-Oz']
EPOCH_DURATION = 30
BATCH_SIZE = 16  # Reduced to mitigate OOM
EPOCHS = 20
SAMPLING_RATE = 50
TELEMETRY_SUBJECTS = [2, 4, 5, 6, 7, 12, 13]
POSSIBLE_HYPNO_LETTERS = 'CHJPUVAEMORW'
print(f"scikit-learn version: {sklearn.__version__}")

nb_classes = 5  # Defined globally for use in functions

def fetch_data(subject_id, night, record_type='PSG'):
    try:
        dataset_id = subject_id
        folder = "sleep-cassette" if night == 1 else "sleep-telemetry"
        if night == 1:
            prefix = f"SC4{dataset_id:02d}"
        else:
            if subject_id not in TELEMETRY_SUBJECTS:
                return None
            telemetry_map = {2: 702, 4: 704, 5: 705, 6: 706, 7: 707, 12: 712, 13: 713}
            prefix = f"ST{telemetry_map.get(subject_id, 700 + dataset_id)}"
        os.makedirs("sleep_edf", exist_ok=True)
        if record_type == 'PSG':
            base_suffix = "E" if night == 1 else "J"
            file_name = f"{prefix}{night if night == 1 else 2}{base_suffix}0-PSG.edf"
            url = f"{BASE_URL}{folder}/{file_name}"
            local_file = os.path.join("sleep_edf", file_name)
            if os.path.exists(local_file):
                return local_file
            urllib.request.urlretrieve(url, local_file)
            print(f"Downloaded {file_name}")
            return local_file
        else:
            base_suffix = "E" if night == 1 else "J"
            for letter in POSSIBLE_HYPNO_LETTERS:
                hypno_suffix = base_suffix + letter
                file_name = f"{prefix}{night if night == 1 else 2}{hypno_suffix}-Hypnogram.edf"
                url = f"{BASE_URL}{folder}/{file_name}"
                local_file = os.path.join("sleep_edf", file_name)
                if os.path.exists(local_file):
                    return local_file
                try:
                    urllib.request.urlretrieve(url, local_file)
                    print(f"Downloaded {file_name}")
                    return local_file
                except urllib.error.HTTPError as e:
                    if e.code != 404:
                        raise
            return None
    except urllib.error.HTTPError as e:
        print(f"HTTP Error {e.code} fetching {file_name if 'file_name' in locals() else 'file'}: {e.reason}")
        return None
    except Exception as e:
        print(f"Error fetching {file_name if 'file_name' in locals() else 'file'}: {e}")
        return None

def get_available_subjects():
    available = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []
        for subject_id in range(NUM_SUBJECTS):
            for night in range(1, NUM_NIGHTS + 1):
                futures.append((
                    subject_id,
                    night,
                    executor.submit(
                        lambda s, n: (
                            fetch_data(s, n, 'PSG') is not None and
                            fetch_data(s, n, 'Hypnogram') is not None
                        ),
                        subject_id, night
                    )
                ))
        for subject_id, night, future in tqdm(futures, desc="Checking availability"):
            if future.result():
                available.append((subject_id, night))
    print(f"Available subject-night pairs: {available}")
    return available

def augment_data(X):
    noise = np.random.normal(0, 0.01, X.shape)
    shift = np.random.randint(-50, 50)
    X_aug = np.roll(X + noise, shift, axis=1)
    return X_aug

def process_subject_night(subject_id, night):
    try:
        psg_file = fetch_data(subject_id, night, 'PSG')
        hypno_file = fetch_data(subject_id, night, 'Hypnogram')
        if psg_file is None or hypno_file is None:
            print(f"Skipping subject {subject_id}, night {night}: Missing files")
            return None, None
        raw = mne.io.read_raw_edf(psg_file, preload=False, verbose=False)
        available_channels = [ch for ch in TARGET_CHANNELS if ch in raw.ch_names]
        if not available_channels:
            print(f"No target channels for subject {subject_id}, night {night}")
            return None, None
        raw.pick_channels(available_channels)
        raw.load_data()
        raw.filter(0.5, 40.0, l_trans_bandwidth=0.5, h_trans_bandwidth=10.0, verbose=False)
        raw.resample(SAMPLING_RATE, npad="auto")
        events = mne.make_fixed_length_events(raw, id=1, duration=EPOCH_DURATION)
        epochs_mne = mne.Epochs(raw, events, tmin=0, tmax=EPOCH_DURATION-1/raw.info['sfreq'],
                                picks=available_channels, baseline=None, preload=True)
        data = epochs_mne.get_data(units='uV')
        annotations = mne.read_annotations(hypno_file)
        labels = np.zeros(len(epochs_mne), dtype=int)
        stage_map = {'Sleep stage W': 0, 'Sleep stage 1': 1, 'Sleep stage 2': 2, 'Sleep stage 3': 3, 'Sleep stage 4': 3, 'Sleep stage R': 4}
        for annot in annotations:
            onset = int(annot['onset'] / EPOCH_DURATION)
            duration = int(annot['duration'] / EPOCH_DURATION)
            stage = annot['description']
            if stage in stage_map:
                for i in range(max(0, onset), min(len(epochs_mne), onset + duration)):
                    labels[i] = stage_map[stage]
        data = (data - np.mean(data, axis=(1, 2), keepdims=True)) / np.std(data, axis=(1, 2), keepdims=True)
        X = data.transpose(0, 2, 1)
        X_aug = augment_data(X)
        X = np.concatenate([X, X_aug])
        labels = np.concatenate([labels, labels])
        del raw, epochs_mne, data
        gc.collect()
        print(f"Processed subject {subject_id}, night {night}: {X.shape[0]} epochs")
        return X, labels
    except Exception as e:
        print(f"Error processing subject {subject_id}, night {night}: {e}")
        return None, None

class DataGenerator(Sequence):
    def __init__(self, X, y, batch_size, augment=True, class_weights=None):
        self.X = X.astype(np.float32)
        self.y = y.astype(np.int32)
        self.batch_size = batch_size
        self.augment = augment
        self.class_weights = class_weights
    def __len__(self):
        return int(np.ceil(len(self.X) / self.batch_size))
    def __getitem__(self, idx):
        start = idx * self.batch_size
        end = min(start + self.batch_size, len(self.X))
        X_batch = self.X[start:end]
        y_batch = self.y[start:end]
        if self.augment:
            X_batch = augment_data(X_batch).astype(np.float32)
        sample_weights = np.ones_like(y_batch, dtype=np.float32)
        if self.class_weights:
            sample_weights = np.array([self.class_weights[label] for label in y_batch], dtype=np.float32)
        return X_batch, y_batch, sample_weights

# Base LSTM Model
def build_lstm_model(input_shape, nb_classes=5):
    inputs = layers.Input(shape=input_shape)
    x = layers.LSTM(128, return_sequences=True)(inputs)
    x = layers.DepthwiseConv1D(input_shape[1], depth_multiplier=2, padding='same', use_bias=False, depthwise_constraint=constraints.max_norm(1.0))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('elu')(x)
    x = layers.MaxPooling1D(pool_size=4)(x)
    x = layers.Dropout(0.25)(x)
    x = layers.SeparableConv1D(64, 16, padding='same', use_bias=False, depthwise_constraint=constraints.max_norm(1.0))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('elu')(x)
    x = layers.MaxPooling1D(pool_size=4)(x)
    x = layers.Dropout(0.25)(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu', name='dense_before_dropout')(x)
    x = layers.Dropout(0.3, name='final_dropout')(x)
    outputs = layers.Dense(nb_classes, activation='softmax', dtype='float32', name='output')(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(0.0005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# EEGNet Model
def build_eegnet_model(input_shape, nb_classes=5, chans=2, samples=1500, dropout_rate=0.25, kern_length=64, f1=8, d=2, f2=16):
    input1 = layers.Input(shape=(samples, chans))
    x = layers.Lambda(lambda t: tf.expand_dims(t, axis=-1))(input1)
    x = layers.Permute((2, 1, 3))(x)

    x = layers.Conv2D(f1, (1, kern_length), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.DepthwiseConv2D((chans, 1), use_bias=False, depth_multiplier=d, depthwise_constraint=constraints.max_norm(1.0))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('elu')(x)
    x = layers.AveragePooling2D((1, 4))(x)
    x = layers.Dropout(dropout_rate)(x)

    x = layers.SeparableConv2D(f2, (1, 16), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('elu')(x)
    x = layers.AveragePooling2D((1, 8))(x)
    x = layers.Dropout(dropout_rate)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu', name='dense_before_dropout')(x)
    x = layers.Dropout(0.3, name='final_dropout')(x)
    outputs = layers.Dense(nb_classes, activation='softmax', dtype='float32', name='output')(x)

    model = models.Model(inputs=input1, outputs=outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(0.0005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Batched prediction function
def batched_predict(model, X, batch_size=BATCH_SIZE, training=False):
    preds = []
    for i in range(0, len(X), batch_size):
        batch = X[i:i+batch_size]
        pred = model(batch, training=training)
        preds.append(pred.numpy())
    return np.concatenate(preds, axis=0)

# Monte Carlo Dropout
def build_mc_dropout_model(base_builder, input_shape, nb_classes=5, dropout_rate=0.25):
    model = base_builder(input_shape, nb_classes)
    return model

def predict_with_mc_dropout(model, X, n_samples=20, batch_size=BATCH_SIZE):
    preds = []
    for _ in range(n_samples):
        pred = batched_predict(model, X, batch_size, training=True)
        preds.append(pred)
    preds = np.stack(preds, axis=0)
    mean_pred = np.mean(preds, axis=0)
    uncertainty = np.std(preds, axis=0)
    return mean_pred, uncertainty

# Deep Ensembles
def train_deep_ensembles(base_builder, input_shape, nb_classes, train_gen, val_gen, n_ensembles=5):
    ensembles = []
    for _ in range(n_ensembles):
        model = base_builder(input_shape, nb_classes)
        model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS, verbose=1)
        ensembles.append(model)
    return ensembles

def predict_with_deep_ensembles(ensembles, X, batch_size=BATCH_SIZE):
    preds = []
    for model in ensembles:
        pred = batched_predict(model, X, batch_size)
        preds.append(pred)
    preds = np.stack(preds, axis=0)
    mean_pred = np.mean(preds, axis=0)
    uncertainty = np.std(preds, axis=0)
    return mean_pred, uncertainty

# Uncertainty Metrics
def negative_log_likelihood(y_true, y_pred):
    y_true_onehot = tf.keras.utils.to_categorical(y_true, num_classes=y_pred.shape[1])
    return -np.mean(np.sum(y_true_onehot * np.log(y_pred + 1e-10), axis=1))

def brier_score(y_true, y_pred):
    y_true_onehot = tf.keras.utils.to_categorical(y_true, num_classes=y_pred.shape[1])
    return np.mean(np.sum((y_pred - y_true_onehot)**2, axis=1))

def expected_calibration_error(y_true, y_pred, num_bins=15):
    pred_classes = np.argmax(y_pred, axis=1)
    prob_max = np.max(y_pred, axis=1)
    correct = (pred_classes == y_true).astype(np.float32)

    bins = np.linspace(0, 1, num_bins + 1)
    indices = np.digitize(prob_max, bins, right=True)

    ece = 0
    for b in range(1, num_bins + 1):
        mask = (indices == b)
        if np.any(mask):
            acc = np.mean(correct[mask])
            conf = np.mean(prob_max[mask])
            ece += np.abs(acc - conf) * np.sum(mask) / len(y_true)
    return ece

def plot_training_curves(history, model_name):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'{model_name} - Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'{model_name} - Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'training_curves_{model_name}.png')
    plt.close()

def evaluate_model(model_name, predict_func, X_test, y_test):
    mean_pred, uncertainty = predict_func(X_test)
    test_acc = np.mean(np.argmax(mean_pred, axis=1) == y_test)
    nll = negative_log_likelihood(y_test, mean_pred)
    brier = brier_score(y_test, mean_pred)
    ece = expected_calibration_error(y_test, mean_pred)

    print(f"\n{model_name} - Test Accuracy: {test_acc:.4f}")
    print(f"{model_name} - NLL: {nll:.4f}")
    print(f"{model_name} - Brier Score: {brier:.4f}")
    print(f"{model_name} - ECE: {ece:.4f}")

    y_pred_classes = np.argmax(mean_pred, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred_classes, average=None)
    stage_names = ['Wake', 'N1', 'N2', 'N3', 'REM']
    print(f"\n{model_name} - Per-class Metrics:")
    for i, stage in enumerate(stage_names):
        print(f"{stage}: Precision={precision[i]:.4f}, Recall={recall[i]:.4f}, F1={f1[i]:.4f}")
    cm = confusion_matrix(y_test, y_pred_classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=stage_names, yticklabels=stage_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'{model_name} - Confusion Matrix')
    plt.savefig(f'confusion_matrix_{model_name}.png')
    plt.close()

    return {'acc': test_acc, 'nll': nll, 'brier': brier, 'ece': ece}

def data_generator(available, batch_size=2000):
    for subject_id, night in available:
        X, y = process_subject_night(subject_id, night)
        if X is None or y is None:
            continue
        for i in range(0, len(X), batch_size):
            yield X[i:i+batch_size], y[i:i+batch_size]
        del X, y
        gc.collect()

def run_pipeline():
    available = get_available_subjects()
    if not available:
        return
    X_train, y_train, X_test, y_test = [], [], [], []
    for X_batch, y_batch in tqdm(data_generator(available), desc="Processing data"):
        if X_batch is None or y_batch is None:
            continue
        class_counts = np.bincount(y_batch)
        stratify = y_batch if min(class_counts[class_counts > 0]) >= 2 else None
        X_tr, X_te, y_tr, y_te = train_test_split(X_batch, y_batch, test_size=0.2, stratify=stratify, random_state=42)
        X_train.append(X_tr); y_train.append(y_tr)
        X_test.append(X_te); y_test.append(y_te)
        del X_batch, y_batch
        gc.collect()
    if not X_train:
        return
    X_train = np.concatenate(X_train)
    y_train = np.concatenate(y_train)
    X_test = np.concatenate(X_test)
    y_test = np.concatenate(y_test)
    class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weight_dict = dict(enumerate(class_weights))
    train_generator = DataGenerator(X_train, y_train, BATCH_SIZE, augment=True, class_weights=class_weight_dict)
    val_generator = DataGenerator(X_test, y_test, BATCH_SIZE, augment=False, class_weights=class_weight_dict)

    input_shape = (X_train.shape[1], X_train.shape[2])

    # Order: EEGNet first, then LSTM; only MC_Dropout and Deep_Ensembles
    base_builders = [('EEGNet', build_eegnet_model), ('LSTM', build_lstm_model)]
    uncertainty_methods = [
        ('MC_Dropout', build_mc_dropout_model, lambda model, X: predict_with_mc_dropout(model, X)),
        ('Deep_Ensembles', train_deep_ensembles, lambda ensembles, X: predict_with_deep_ensembles(ensembles, X))
    ]

    results = {}
    for base_name, base_builder in base_builders:
        for unc_name, build_func, predict_func in uncertainty_methods:
            model_name = f"{base_name}_{unc_name}"
            print(f"\nTraining {model_name}...")
            if unc_name == 'Deep_Ensembles':
                models = build_func(base_builder, input_shape, nb_classes, train_generator, val_generator)
                metrics = evaluate_model(model_name, lambda X: predict_func(models, X), X_test, y_test)
            else:
                model = build_func(base_builder, input_shape, nb_classes)
                history = model.fit(train_generator, validation_data=val_generator, epochs=EPOCHS, verbose=1)
                plot_training_curves(history, model_name)
                metrics = evaluate_model(model_name, lambda X: predict_func(model, X), X_test, y_test)
            results[model_name] = metrics

    # Compare
    print("\nModel Comparison for Uncertainty Estimation:")
    print("Sorted by ECE (lower is better):")
    for name, met in sorted(results.items(), key=lambda x: x[1]['ece']):
        print(f"{name}: Acc={met['acc']:.4f}, NLL={met['nll']:.4f}, Brier={met['brier']:.4f}, ECE={met['ece']:.4f}")

    best_model = min(results, key=lambda k: results[k]['ece'])
    print(f"\nBest fit model for uncertainty estimation: {best_model} with ECE {results[best_model]['ece']:.4f}")

if __name__ == "__main__":
    run_pipeline()

In [ ]:
Training EEGNet_MC_Dropout...
I0000 00:00:1753153391.092152    6133 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9797 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:d9:00.0, compute capability: 7.5
/home/samibc/miniconda3/envs/tf-mnist/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
Epoch 1/3
WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
I0000 00:00:1753153394.378154    6409 service.cc:148] XLA service 0x7f0810001460 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753153394.378195    6409 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2025-07-21 23:03:14.432101: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1753153394.769836    6409 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-07-21 23:03:15.632136: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k2=0,k4=2,k5=3,k6=3,k7=2} for conv (f16[16,1,375,16]{3,2,1,0}, u8[0]{0}) custom-call(f16[16,1,375,16]{3,2,1,0}, f16[16,1,1,16]{3,2,1,0}), window={size=1x1}, dim_labels=b01f_o01i->b01f, custom_call_target="__cudnn$convForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2025-07-21 23:03:15.635230: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k2=4,k4=1,k5=4,k6=3,k7=2} for conv (f16[16,1,375,16]{3,2,1,0}, u8[0]{0}) custom-call(f16[16,1,375,16]{3,2,1,0}, f16[16,1,1,16]{3,2,1,0}), window={size=1x1}, dim_labels=b01f_o01i->b01f, custom_call_target="__cudnn$convForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
I0000 00:00:1753153398.806185    6409 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
4548/5956 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.5825 - loss: 1.15112025-07-21 23:03:36.506757: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k2=0,k4=2,k5=3,k6=3,k7=2} for conv (f16[10,1,375,16]{3,2,1,0}, u8[0]{0}) custom-call(f16[10,1,375,16]{3,2,1,0}, f16[16,1,1,16]{3,2,1,0}), window={size=1x1}, dim_labels=b01f_o01i->b01f, custom_call_target="__cudnn$convForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2025-07-21 23:03:36.507738: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k2=4,k4=1,k5=4,k6=3,k7=2} for conv (f16[10,1,375,16]{3,2,1,0}, u8[0]{0}) custom-call(f16[10,1,375,16]{3,2,1,0}, f16[16,1,1,16]{3,2,1,0}), window={size=1x1}, dim_labels=b01f_o01i->b01f, custom_call_target="__cudnn$convForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
5954/5956 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6095 - loss: 1.09412025-07-21 23:03:49.856945: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k2=0,k4=2,k5=3,k6=3,k7=2} for conv (f16[8,1,375,16]{3,2,1,0}, u8[0]{0}) custom-call(f16[8,1,375,16]{3,2,1,0}, f16[16,1,1,16]{3,2,1,0}), window={size=1x1}, dim_labels=b01f_o01i->b01f, custom_call_target="__cudnn$convForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2025-07-21 23:03:49.858095: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k2=4,k4=1,k5=4,k6=3,k7=2} for conv (f16[8,1,375,16]{3,2,1,0}, u8[0]{0}) custom-call(f16[8,1,375,16]{3,2,1,0}, f16[16,1,1,16]{3,2,1,0}), window={size=1x1}, dim_labels=b01f_o01i->b01f, custom_call_target="__cudnn$convForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 38s 5ms/step - accuracy: 0.6096 - loss: 1.0940 - val_accuracy: 0.8017 - val_loss: 0.6365
Epoch 2/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - accuracy: 0.7868 - loss: 0.6856 - val_accuracy: 0.8089 - val_loss: 0.6215
Epoch 3/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 26s 4ms/step - accuracy: 0.8108 - loss: 0.6297 - val_accuracy: 0.8284 - val_loss: 0.5404

EEGNet_MC_Dropout - Test Accuracy: 0.8463
EEGNet_MC_Dropout - NLL: 0.4383
EEGNet_MC_Dropout - Brier Score: 0.2255
EEGNet_MC_Dropout - ECE: 0.0353

EEGNet_MC_Dropout - Per-class Metrics:
Wake: Precision=0.9864, Recall=0.8959, F1=0.9389
N1: Precision=0.2228, Recall=0.6906, F1=0.3369
N2: Precision=0.8799, Recall=0.7286, F1=0.7971
N3: Precision=0.7122, Recall=0.9196, F1=0.8027
REM: Precision=0.6984, Recall=0.7759, F1=0.7351

Training EEGNet_Deep_Ensembles...
Epoch 1/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 36s 5ms/step - accuracy: 0.6322 - loss: 1.0842 - val_accuracy: 0.7611 - val_loss: 0.6892
Epoch 2/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 27s 5ms/step - accuracy: 0.7906 - loss: 0.6781 - val_accuracy: 0.5772 - val_loss: 0.8797
Epoch 3/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - accuracy: 0.8112 - loss: 0.6333 - val_accuracy: 0.8503 - val_loss: 0.5907
Epoch 1/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 34s 5ms/step - accuracy: 0.6611 - loss: 1.0459 - val_accuracy: 0.7380 - val_loss: 0.7157
Epoch 2/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - accuracy: 0.7906 - loss: 0.7020 - val_accuracy: 0.7458 - val_loss: 0.6370
Epoch 3/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 26s 4ms/step - accuracy: 0.8135 - loss: 0.6422 - val_accuracy: 0.7858 - val_loss: 0.6337
Epoch 1/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 36s 5ms/step - accuracy: 0.6360 - loss: 1.0716 - val_accuracy: 0.7758 - val_loss: 0.7061
Epoch 2/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - accuracy: 0.7903 - loss: 0.6969 - val_accuracy: 0.8100 - val_loss: 0.6033
Epoch 3/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - accuracy: 0.8118 - loss: 0.6394 - val_accuracy: 0.8058 - val_loss: 0.5692
Epoch 1/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 37s 5ms/step - accuracy: 0.5927 - loss: 1.0828 - val_accuracy: 0.7340 - val_loss: 0.6682
Epoch 2/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - accuracy: 0.7916 - loss: 0.6852 - val_accuracy: 0.6433 - val_loss: 0.7293
Epoch 3/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - accuracy: 0.8165 - loss: 0.6205 - val_accuracy: 0.8447 - val_loss: 0.5329
Epoch 1/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 40s 6ms/step - accuracy: 0.6149 - loss: 1.0802 - val_accuracy: 0.7641 - val_loss: 0.6554
Epoch 2/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - accuracy: 0.7978 - loss: 0.6764 - val_accuracy: 0.7815 - val_loss: 0.6150
Epoch 3/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 27s 4ms/step - accuracy: 0.8184 - loss: 0.6248 - val_accuracy: 0.7319 - val_loss: 0.6607

EEGNet_Deep_Ensembles - Test Accuracy: 0.8232
EEGNet_Deep_Ensembles - NLL: 0.4918
EEGNet_Deep_Ensembles - Brier Score: 0.2540
EEGNet_Deep_Ensembles - ECE: 0.0139

EEGNet_Deep_Ensembles - Per-class Metrics:
Wake: Precision=0.9979, Recall=0.8707, F1=0.9300
N1: Precision=0.2104, Recall=0.6821, F1=0.3216
N2: Precision=0.9422, Recall=0.6683, F1=0.7820
N3: Precision=0.5856, Recall=0.9682, F1=0.7298
REM: Precision=0.6028, Recall=0.7961, F1=0.6861

Training LSTM_MC_Dropout...
Epoch 1/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 312s 51ms/step - accuracy: 0.7482 - loss: 0.8370 - val_accuracy: 0.7641 - val_loss: 0.7784
Epoch 2/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 304s 51ms/step - accuracy: 0.8730 - loss: 0.5152 - val_accuracy: 0.8663 - val_loss: 0.5133
Epoch 3/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 305s 51ms/step - accuracy: 0.8822 - loss: 0.4706 - val_accuracy: 0.8793 - val_loss: 0.4722

LSTM_MC_Dropout - Test Accuracy: 0.8996
LSTM_MC_Dropout - NLL: 0.2788
LSTM_MC_Dropout - Brier Score: 0.1454
LSTM_MC_Dropout - ECE: 0.0066

LSTM_MC_Dropout - Per-class Metrics:
Wake: Precision=0.9975, Recall=0.9509, F1=0.9736
N1: Precision=0.2974, Recall=0.7482, F1=0.4256
N2: Precision=0.9245, Recall=0.8059, F1=0.8611
N3: Precision=0.8003, Recall=0.8996, F1=0.8471
REM: Precision=0.7989, Recall=0.8143, F1=0.8065

Training LSTM_Deep_Ensembles...
Epoch 1/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 308s 51ms/step - accuracy: 0.7446 - loss: 0.8568 - val_accuracy: 0.7630 - val_loss: 0.6655
Epoch 2/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 305s 51ms/step - accuracy: 0.8702 - loss: 0.5252 - val_accuracy: 0.8556 - val_loss: 0.5541
Epoch 3/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 305s 51ms/step - accuracy: 0.8838 - loss: 0.4672 - val_accuracy: 0.8541 - val_loss: 0.5783
Epoch 1/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 309s 51ms/step - accuracy: 0.7526 - loss: 0.8315 - val_accuracy: 0.8324 - val_loss: 0.5371
Epoch 2/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 305s 51ms/step - accuracy: 0.8674 - loss: 0.5267 - val_accuracy: 0.7759 - val_loss: 1.0527
Epoch 3/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 305s 51ms/step - accuracy: 0.8818 - loss: 0.4727 - val_accuracy: 0.8897 - val_loss: 0.4659
Epoch 1/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 309s 51ms/step - accuracy: 0.7515 - loss: 0.8356 - val_accuracy: 0.8123 - val_loss: 0.6094
Epoch 2/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 306s 51ms/step - accuracy: 0.8657 - loss: 0.5440 - val_accuracy: 0.8212 - val_loss: 0.5395
Epoch 3/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 305s 51ms/step - accuracy: 0.8818 - loss: 0.4654 - val_accuracy: 0.8627 - val_loss: 0.5120
Epoch 1/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 308s 51ms/step - accuracy: 0.7442 - loss: 0.8509 - val_accuracy: 0.8199 - val_loss: 0.7179
Epoch 2/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 305s 51ms/step - accuracy: 0.8646 - loss: 0.5434 - val_accuracy: 0.8634 - val_loss: 0.5262
Epoch 3/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 305s 51ms/step - accuracy: 0.8826 - loss: 0.4723 - val_accuracy: 0.8479 - val_loss: 0.5534
Epoch 1/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 306s 51ms/step - accuracy: 0.7572 - loss: 0.8432 - val_accuracy: 0.8061 - val_loss: 0.8317
Epoch 2/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 302s 51ms/step - accuracy: 0.8682 - loss: 0.5334 - val_accuracy: 0.8296 - val_loss: 0.5420
Epoch 3/3
5956/5956 ━━━━━━━━━━━━━━━━━━━━ 302s 51ms/step - accuracy: 0.8801 - loss: 0.4823 - val_accuracy: 0.8677 - val_loss: 0.5478

LSTM_Deep_Ensembles - Test Accuracy: 0.8862
LSTM_Deep_Ensembles - NLL: 0.3105
LSTM_Deep_Ensembles - Brier Score: 0.1623
LSTM_Deep_Ensembles - ECE: 0.0124

LSTM_Deep_Ensembles - Per-class Metrics:
Wake: Precision=0.9987, Recall=0.9382, F1=0.9675
N1: Precision=0.2834, Recall=0.7764, F1=0.4152
N2: Precision=0.9268, Recall=0.7970, F1=0.8570
N3: Precision=0.7546, Recall=0.9146, F1=0.8269
REM: Precision=0.7484, Recall=0.7473, F1=0.7478

Model Comparison for Uncertainty Estimation:
Sorted by ECE (lower is better):
LSTM_MC_Dropout: Acc=0.8996, NLL=0.2788, Brier=0.1454, ECE=0.0066
LSTM_Deep_Ensembles: Acc=0.8862, NLL=0.3105, Brier=0.1623, ECE=0.0124
EEGNet_Deep_Ensembles: Acc=0.8232, NLL=0.4918, Brier=0.2540, ECE=0.0139
EEGNet_MC_Dropout: Acc=0.8463, NLL=0.4383, Brier=0.2255, ECE=0.0353

Best fit model for uncertainty estimation: LSTM_MC_Dropout with ECE 0.0066
(tf-mnist) Tue Jul 22, 01:25:30 [samibc@cs02 /deepLearning/samibc/sleep]$ client_loop: send disconnect: Connection reset
PS C:\Users\bcsam>
